In [17]:
import re, os, os.path
import requests

from urllib.parse import urljoin
from bs4 import BeautifulSoup

def _not_num(n):
    try: 
        n = float(n)
        return False
    except: 
        return True

def _is_num(n):
    try: 
        n = float(n)
        return True
    except: 
        return False
    

def find_first_num(list):
    for i in list:
        try:
            i = i.replace(',', '')
            i = i.replace('$', '')
            i = int(i)
        except:
            pass
        if _is_num(i):
            return i
        

def html_to_list(html_file):        
    with open(html_file) as file:
        soup = BeautifulSoup(file, 'html.parser')
    
    locator = 'div table tr td p'
    dollar_regex = '[0-9]*,*[0-9]*,*[0-9]+'

    item = soup.select(locator)
    lines = []

    for i in item:
        if i.string is None:
            pass
        elif i.contents[0] == '\n':
            pass
        else:
            line = str(i.contents[0]) #convert single element list to string
            line = line.replace('\n', '').replace('  ', '').replace('(', '').replace(')', '').replace(u'\xa0', u' ') #strip string characters
            lines.append(line.strip())
        
    while '' in lines:
        lines.remove('')
    while '—' in lines:
        lines.remove('—')
    while '$' in lines:
        lines.remove('$')
    return lines

def filter_monies(string):
    dollar_regex = '[0-9]*,*[0-9]*,*[0-9]+'
    match = re.match(dollar_regex, string)
    if match:
        value = match.group()
        return value.replace(',','')
    return string

def filter_year(string):
    year_regex = '2[0-9][0-9][0-9]'  #any year between 2000 - 2999
    match = re.match(year_regex, string)
    if match:
        value = 'Year:' + match.group() #add to year to make string != number
        return value
    return string


def structure(list, starter=1): #starter should be the string you want to the structured table to start with
    if starter == 1:
        search_set = {'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'}
    else:
        search_set = set(starter.split(' '))
        
    index = 0
    for element in list:  #
        element.replace(u'\xa0', u' ')
        comp = set(element.split(' '))  #decompose element into a set of strings
        if comp.intersection(search_set) and starter == 1:
            print('*************************found starting condition.')
            del list[:index]
            break
        elif len(comp.intersection(search_set)) == len(search_set):
            del list[:index]
            break
        index += 1
    return list

s = html_to_list('1.html')
S = structure(s)

print(S)

*************************found starting condition.
['March 31,', 'December 31,', '2019', '2018', 'Assets', 'Current assets', 'Cash and cash equivalents', '2,198,169', '3,685,618', 'Restricted cash', '130,950', '192,551', 'Accounts receivable, net', '1,046,945', '949,022', 'Inventory', '3,836,850', '3,113,446', 'Prepaid expenses and other current assets', '464,908', '365,671', 'Total current assets', '7,677,822', '8,306,308', 'Operating lease vehicles, net', '1,972,502', '2,089,758', 'Solar energy systems, net', '6,241,637', '6,271,396', 'Property, plant and equipment, net', '9,850,929', '11,330,077', 'Operating lease right-of-use assets', '1,253,027', 'Intangible assets, net', '273,568', '282,492', 'Goodwill', '74,312', '68,159', 'MyPower customer notes receivable, net of current portion', '413,181', '421,548', 'Restricted cash, net of current portion', '353,679', '398,219', 'Other assets', '801,867', '571,657', 'Total assets', '28,912,524', '29,739,614', 'Liabilities', 'Current liabil

In [19]:
string = ['March\xa031,', 'December 31,', '2019', '2018', 'Assets', 'Current assets', 'Cash and cash equivalents', '2,198,169', '3,685,618']

def structureA(list):
    key_temp = []
    val_temp = []
    for i in list:
        i = filter_year(i)  #search for year first to avoid matching with dollar amounts
        i = filter_monies(i)
        if _not_num(i):
            #print(f'{i} is not a number')
            key_temp.append(i)
        elif _is_num(i):
            #print(f'{i} is a number')
            val_temp.append(i)
    return dict(zip(key_temp, val_temp))
'''
b = S[:]
del b[:3]

B = structure(b)
del B[:3]
C = structure(B)
'''

A = structure(s, ('Cash and cash equivalents'))
print(A)



['Cash and cash equivalents', '2,198,169', '3,685,618', 'Restricted cash', '130,950', '192,551', 'Accounts receivable, net', '1,046,945', '949,022', 'Inventory', '3,836,850', '3,113,446', 'Prepaid expenses and other current assets', '464,908', '365,671', 'Total current assets', '7,677,822', '8,306,308', 'Operating lease vehicles, net', '1,972,502', '2,089,758', 'Solar energy systems, net', '6,241,637', '6,271,396', 'Property, plant and equipment, net', '9,850,929', '11,330,077', 'Operating lease right-of-use assets', '1,253,027', 'Intangible assets, net', '273,568', '282,492', 'Goodwill', '74,312', '68,159', 'MyPower customer notes receivable, net of current portion', '413,181', '421,548', 'Restricted cash, net of current portion', '353,679', '398,219', 'Other assets', '801,867', '571,657', 'Total assets', '28,912,524', '29,739,614', 'Liabilities', 'Current liabilities', 'Accounts payable', '3,248,827', '3,404,451', 'Accrued liabilities and other', '2,276,951', '2,094,253', 'Deferred r

In [83]:
strings = [
    'March\xa031,',
    'Three\xa0Months\xa0Ended\xa0March\xa031, 2019',
    'March 31',
    'Three Months Ended March 31',
    'Three Months Ended March 31, 2018'
]


reg = '(\S)+'

for i in strings:
    match = re.search(reg, i)
    print(match.group(0))

March
Three
March
Three
Three


In [30]:
months = {'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'}

string = ['March\xa031,', 'December 31,', '2019', '2018', 'Assets', 'Current assets', 'Cash and cash equivalents', '2,198,169', '3,685,618']
year_regex = '2[0-9][0-9][0-9]'  #any year between 2000 - 2999
month_day_re = '\w+\[a-z]+[0-9]*[0-3][0-9]'

MM = []
YYYY = []
total = []
for item in string:
    
    try: 
        match = re.match(year_regex, item)
    except:
        pass
    i = set(item.split(' '))
    
    if i.intersection(months):
        month_match = list(i.intersection(months))
        print(f'found month: {month_match} ')
        MM.append(month_match[0])
    elif match:
        print(f'found year: {item}')
        YYYY.append(item)
    

for l in range(len(MM)):    
    total.append(MM[l]+YYYY[l])
   
    

print(MM)
print(YYYY)
dict(zip(MM, YYYY))
        
print(total)

found month: ['December'] 
found year: 2019
found year: 2018
['December']
['2019', '2018']
['December2019']


In [62]:
test = 'March 31'
month_day_re = '(\S)*'

a = re.search(month_day_re, test)
a = a.group(0)
print(a)

A = {a}
print(A)
if A.intersection(months):
    print('match')

March
{'March'}
match


In [16]:
test = 'Three Months Ended March 31, 2019'
s = {'March', 'other', 'strings'}
r = ('March Ended Three')
R = set(r.split(' '))
print(R)


p = set(test.split(' '))

if len(p.intersection(R)) == len(R):
    print('iii')
    
print(p.intersection(s))

{'March', 'Ended', 'Three'}
iii
{'March'}


In [89]:
clear()